In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import pymongo 
import requests

starbucks 사이트에서 시도 목록을 얻어와서 starbucks 데이터베이스의 sido 컬렉션에 저장한다.

In [2]:
mongoConnection = pymongo.MongoClient('mongodb://localhost:27017')
starbucks = mongoConnection['starbucks']
sido = starbucks['sido']
sido.delete_many({})

request = requests.post('https://www.starbucks.co.kr/store/getSidoList.do').json()
sidoList = []
for r in request['list']:
    d = {'sido_cd': r['sido_cd'].strip(), 'sido_nm': r.get('sido_nm').strip()}
    sidoList.append(d)
    
for s in sidoList:
    sido.insert_one(s)
    
sidoList = sido.find({}, {'_id':0})
for s in sidoList:
    print('[{}]'.format(s['sido_nm']), end=' ')

[서울] [경기] [광주] [대구] [대전] [부산] [울산] [인천] [강원] [경남] [경북] [전남] [전북] [충남] [충북] [제주] [세종] 

시도 이름을 입력받아 시도의 구군 코드 정보를 얻어와서 starbucks 데이터베이스의 gugun 컬렉션에 저장한다.

In [3]:
sidoName = input('구군 코드를 검색하려는 시도 이름을 입력하세요(예: 서울): ')
sido_cd = sido.find({'sido_nm': sidoName}, {'_id': 0, 'sido_cd': 1, 'sid o_nm': 1}).next()
# print(sido_cd['sido_cd'])

request = requests.post('https://www.starbucks.co.kr/store/getGugunList.do', data={
    'sido_cd': sido_cd['sido_cd']
}).json()
gugunList = []
for r in request['list']:
    if r['gugun_nm'] is None: # 세종 None 처리
        r['gugun_nm'] = '세부지역없음'
    d = {'gugun_cd': r['gugun_cd'].strip(), 'gugun_nm': r['gugun_nm'].strip()}
    gugunList.append(d)
# print(gugunList)

gugun = starbucks['gugun'] # gugun 컬렉션을 만든다.
gugun.delete_many({}) # gugun 컬렉션에 저장된 모든 데이터를 제거한다.
for g in gugunList:
    gugun.insert_one(g)
    
gugunList = gugun.find({}, {'_id': 0, 'gugun_cd': 1, 'gugun_nm': 1})
for s in gugunList:
    print('[{}]'.format(s['gugun_nm']), end=' ')

구군 코드를 검색하려는 시도 이름을 입력하세요(예: 서울): 서울
[강남구] [강동구] [강북구] [강서구] [관악구] [광진구] [구로구] [금천구] [노원구] [도봉구] [동대문구] [동작구] [마포구] [서대문구] [서초구] [성동구] [성북구] [송파구] [양천구] [영등포구] [용산구] [은평구] [종로구] [중구] [중랑구] 

시도 이름과 구군 이름을 입력받아 매장 정보를 얻어와서 starbucks 데이터베이스의 store 컬렉션에 저장한다.

In [8]:
sidoList = sido.find({})
for s in sidoList:
    print('[{}]'.format(s['sido_nm']), end=' ')
sidoName = input('구군 코드를 검색하려는 시도 이름을 입력하세요(예: 서울): ')

gugunLust = gugun.find({}, {'_id': 0})
for g in gugunList:
    print('[{}]'.format(g['gugun_nm']), end=' ')

[서울] [경기] [광주] [대구] [대전] [부산] [울산] [인천] [강원] [경남] [경북] [전남] [전북] [충남] [충북] [제주] [세종] 구군 코드를 검색하려는 시도 이름을 입력하세요(예: 서울): 서울


시도 이름과 구군 이름을 입력받아 매장 정보를 얻어와서 starbucks 데이터베이스의 store 컬렉션에 저장한다.

In [11]:
# sidoName = input('구군코드를 검색하려는 시도 이름을 입력하세요(예: 서울): ')
# sido_cd = sido.find({'sido_nm': sidoName}, {'_id': 0, 'sido_cd': 1, 'sido_nm': 1}).next()
gugunName = input('구군별 매장을 검색하려는 구군 이름을 입력하세요(예: 가평군): ')
gugun_cd = gugun.find({'gugun_nm': gugunName}, {'_id': 0, 'gugun_cd': 1, 'gugun_nm': 1}).next()
# print(gugun_cd['gugun_cd'])

request = requests.post('https://www.starbucks.co.kr/store/getStore.do?r=XBOGDH6O1Z', data={
    'ins_lat': 37.5750656,
    'ins_lng': 126.9694464,
    'p_sido_cd': '',
    'p_gugun_cd': gugun_cd['gugun_cd'],
    'in_biz_cd': '',
    'iend': 2000,
    'set_date': ''
}).json()
storeList = []
for r in request['list']:
    d = {'s_name': r['s_name'].strip(), 'doro_address': r['doro_address'].strip(), 'lat': r['lat'], 'lot': r['lot']}
    storeList.append(d)
# print(storeList)

store = starbucks['store'] # store 컬렉션을 만든다.
store.delete_many({}) # store 컬렉션에 저장된 모든 데이터를 제거한다.
for s in storeList:
    store.insert_one(s)
    
storeList = store.find({}, {'_id': 0})
for s in storeList:
    print(s)

구군별 매장을 검색하려는 구군 이름을 입력하세요(예: 가평군): 강남구
{'s_name': '역삼이마트', 'doro_address': '서울특별시 강남구 역삼로 310 (역삼동)', 'lat': '37.499367', 'lot': '127.048425'}
{'s_name': '삼성동', 'doro_address': '서울특별시 강남구 영동대로96길 12 (삼성동)', 'lat': '37.510843366121534', 'lot': '127.06363342044263'}
{'s_name': '강남논현', 'doro_address': '서울특별시 강남구 강남대로 512 (논현동)', 'lat': '37.5078978596254', 'lot': '127.023338614644'}
{'s_name': '테헤란로아남타워', 'doro_address': '서울특별시 강남구 테헤란로 311 (역삼동)', 'lat': '37.50383', 'lot': '127.04487'}
{'s_name': '삼성역', 'doro_address': '서울특별시 강남구 테헤란로 620 (대치동)', 'lat': '37.509206962541256', 'lot': '127.06560851480327'}
{'s_name': '강남역신분당역사', 'doro_address': '서울특별시 강남구 강남대로 396 (역삼동)', 'lat': '37.4970572543978', 'lot': '127.028180714381'}
{'s_name': '언주역', 'doro_address': '서울특별시 강남구 봉은사로 222 (역삼동)', 'lat': '37.507775', 'lot': '127.036472'}
{'s_name': '대치삼성프라자', 'doro_address': '서울특별시 강남구 영동대로 237 (대치동)', 'lat': '37.500313', 'lot': '127.066961'}
{'s_name': '강남세곡', 'doro_address': '서울특별시 강남구 헌릉로569길 18 (세곡

시도 이름을 입력받아 시도 전체 매장 정보를 starbucks 데이터베이스의 store_sido 컬렉션에 저장한다.

In [19]:
sidoName = input('구군코드를 검색하려는 시도 이름을 입력하세요(예: 서울): ')
sido_cd: sido.find({'sido_nm': sidoName}, {'_id': 0}).next()
print(sido_cd)

request = requests.post('https://www.starbucks.co.kr/store/getStore.do?r=XBOGDH6O1Z', 
            data={
                'ins_lat': 37.5750656,
                'ins_lng': 126.9694464,
                'p_sido_cd': '',
                'p_gugun_cd': gugun_cd['gugun_cd'],
                'in_biz_cd': '',
                'iend': 2000,
                'set_date': ''
            }).json()
storeList = []
for r in request['list']:
    d = {'s_name': s['s_name'].strip(), 'doro_address': s['doro_address'].strip(), 'lat': r['lat'], 'lot': r['lot']}
    # print(d)
    storeList.append(d)
# print(storeList)

store_sido = starbucks['store_sido']
store_sido.delete_many({})
for s in storeList:
    print(s)

구군코드를 검색하려는 시도 이름을 입력하세요(예: 서울): 세종
{'sido_cd': '01'}
{'s_name': '역삼아레나빌딩', 'doro_address': '서울특별시 강남구 언주로 425 (역삼동)', 'lat': '37.499367', 'lot': '127.048425'}
{'s_name': '역삼아레나빌딩', 'doro_address': '서울특별시 강남구 언주로 425 (역삼동)', 'lat': '37.510843366121534', 'lot': '127.06363342044263'}
{'s_name': '역삼아레나빌딩', 'doro_address': '서울특별시 강남구 언주로 425 (역삼동)', 'lat': '37.5078978596254', 'lot': '127.023338614644'}
{'s_name': '역삼아레나빌딩', 'doro_address': '서울특별시 강남구 언주로 425 (역삼동)', 'lat': '37.50383', 'lot': '127.04487'}
{'s_name': '역삼아레나빌딩', 'doro_address': '서울특별시 강남구 언주로 425 (역삼동)', 'lat': '37.509206962541256', 'lot': '127.06560851480327'}
{'s_name': '역삼아레나빌딩', 'doro_address': '서울특별시 강남구 언주로 425 (역삼동)', 'lat': '37.4970572543978', 'lot': '127.028180714381'}
{'s_name': '역삼아레나빌딩', 'doro_address': '서울특별시 강남구 언주로 425 (역삼동)', 'lat': '37.507775', 'lot': '127.036472'}
{'s_name': '역삼아레나빌딩', 'doro_address': '서울특별시 강남구 언주로 425 (역삼동)', 'lat': '37.500313', 'lot': '127.066961'}
{'s_name': '역삼아레나빌딩', 'doro_address': '서

전국 전체 매장 정보를 starbucks 데이터베이스의 store_all 컬렉션에 저장한다.

In [21]:
request = requests.post('https://www.starbucks.co.kr/store/getStore.do?r=XBOGDH6O1Z', 
            data={
                'ins_lat': 37.5750656,
                'ins_lng': 126.9694464,
                'p_sido_cd': '',
                'p_gugun_cd': '',
                'in_biz_cd': '',
                'iend': 2000,
                'set_date': ''
            }).json()
storeList = []
for r in request['list']:
    d = {'s_name': s['s_name'].strip(), 'doro_address': s['doro_address'].strip(), 'lat': r['lat'], 'lot': r['lot']}
    storeList.append(d)

store_all = starbucks['store_all']
store_all.delete_many({})
for s in storeList:
    store_all.insert_one(s)
    
storeList = store_all.find({}, {'_id': 0})
for s in storeList:
    print(s)

{'s_name': '역삼아레나빌딩', 'doro_address': '서울특별시 강남구 언주로 425 (역삼동)', 'lat': '36.1069986', 'lot': '128.4207103'}
{'s_name': '역삼아레나빌딩', 'doro_address': '서울특별시 강남구 언주로 425 (역삼동)', 'lat': '37.557175', 'lot': '126.923654'}
{'s_name': '역삼아레나빌딩', 'doro_address': '서울특별시 강남구 언주로 425 (역삼동)', 'lat': '33.481794', 'lot': '126.482523'}
{'s_name': '역삼아레나빌딩', 'doro_address': '서울특별시 강남구 언주로 425 (역삼동)', 'lat': '37.499829', 'lot': '127.147672'}
{'s_name': '역삼아레나빌딩', 'doro_address': '서울특별시 강남구 언주로 425 (역삼동)', 'lat': '35.1600338', 'lot': '129.1630512'}
{'s_name': '역삼아레나빌딩', 'doro_address': '서울특별시 강남구 언주로 425 (역삼동)', 'lat': '37.518807', 'lot': '127.00704'}
{'s_name': '역삼아레나빌딩', 'doro_address': '서울특별시 강남구 언주로 425 (역삼동)', 'lat': '37.560809', 'lot': '126.986397'}
{'s_name': '역삼아레나빌딩', 'doro_address': '서울특별시 강남구 언주로 425 (역삼동)', 'lat': '35.16981023', 'lot': '129.12778297'}
{'s_name': '역삼아레나빌딩', 'doro_address': '서울특별시 강남구 언주로 425 (역삼동)', 'lat': '37.3824512554796', 'lot': '126.636816667557'}
{'s_name': '역삼아레나빌딩', 'dor